# RAG para responder perguntas de PDF

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from chromadb.config import Settings
import chromadb
import re

### Carregando páginas do PDF

In [2]:
loader = PyPDFLoader('WD10M44530W_03786S-00_BPT.pdf')
pages = [page async for page in loader.alazy_load()]


In [7]:
pages[0].metadata

{'source': 'WD10M44530W_03786S-00_BPT.pdf', 'page': 0}

### Function para limpeza de texto
Remoção de caracteres em branco e especiais

In [3]:
def clean_text(text:str):
    ct = text.strip()
    ct = re.sub(r'\s+', ' ', ct)
    ct = re.sub(r'[^\w\s]','', ct)
    return ct

In [4]:
for page in pages:
    text = clean_text(page.page_content)
    page.page_content = text

### Divisão do documento em blocos de 500 palavras

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=len)
docs = splitter.transform_documents(pages)

In [14]:
c_page = 0
cid = 0
for d in docs:
    if d.metadata['page'] > c_page:
        c_page = d.metadata['page']
        cid = 0
    d.metadata = {**d.metadata, 'cid':cid}
    cid +=1

In [30]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]

### Adicionando documentos na vector store (banco de dados de vetores)
**OBS.: deve alterar para usar sua chave da API da OpenAI**

In [6]:
client = chromadb.HttpClient(settings=Settings(allow_reset=True))
embeddings = OpenAIEmbeddings(api_key='chave openAI aqui')

vector_store = Chroma(collection_name='manual_maq', client=client, embedding_function=embeddings)

In [32]:
vector_store.add_documents(documents=docs, ids=uuids)

['db3a8fe5-f029-475c-b2f8-aba2e7e52b88',
 'af8354d5-d499-4e50-8451-6c3221f2db91',
 '64d85a3e-a4d1-473b-82bf-a28e6466295e',
 'b6df8021-1ecc-4bc7-9eb7-0b09562f14d9',
 'f5ff10b8-f086-4047-a1d7-7d161c8b76e9',
 'd52eeb6f-c0ef-45e1-a6cf-a0fd1e5ea78e',
 'd8827561-6a99-4903-9efc-b52d46d74259',
 '33668887-fffa-468f-8211-e7c9bc8ff9a7',
 'a9f9aa42-4912-4a45-be23-746993702532',
 'e2204e94-3df5-4930-aa41-eb774c7e72f8',
 '7a1fa2ad-0ff0-42e7-ab28-f86c088502f6',
 '41899731-9615-435d-84b0-a8925ffef60f',
 '8cc09530-2272-4625-b43d-daa0b07e5658',
 '817de10c-9bac-4afe-be90-d3545bfa821a',
 '63cae1d2-09d4-45e0-82f4-3096544b9e0a',
 '9c32d221-ebf2-4da5-97f6-91b10fdcb187',
 '866e6ccc-00fb-4b18-937d-b2236a57860d',
 'c2a3c97b-d306-46a4-9e17-e1a28a5d8059',
 '3aba46f6-687d-4f59-9756-c105f70ec323',
 '60782c43-b564-4c46-b08a-f0a4af69ea08',
 '3f337226-0df3-4319-a86d-635674298e1e',
 '87279cf4-1d56-4d72-9905-a987781ab7a5',
 '687bef00-1c36-490b-8567-87b674523fea',
 '105cfec5-f3e9-479d-b582-385b58baf74e',
 '06758738-7c11-

### Criação de prompt para enviar para o modelo LLM

In [8]:
template = ChatPromptTemplate([
    ('system', ''' Preciso de ajuda para analisar o texto a seguir. Eu farei perguntas baseadas no conteúdo e você deve usar o contexto fornecido abaixo para dar a resposta:
     {contexto} '''),
    ('user', 'Usando o contexto fornecido, responda a pergunta: {pergunta}')
])

template.invoke({'contexto': 'teste', 'pergunta': 'mensagem'})

ChatPromptValue(messages=[SystemMessage(content=' Preciso de ajuda para analisar o texto a seguir. Eu farei perguntas baseadas no conteúdo e você deve usar o contexto fornecido abaixo para dar a resposta:\n     teste ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Usando o contexto fornecido, responda a pergunta: mensagem', additional_kwargs={}, response_metadata={})])

In [10]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0, api_key='chave openAI aqui')
chain = template | llm

### Busca de contexto na vector store baseado na similaridade do texto da pergunta
Irá devolver 10 registros (**k=10**)

In [ ]:
context = vector_store.similarity_search('como devo fazer para realizar a secagem de roupas', k=10)
context

[Document(metadata={'cid': 0, 'page': 32, 'source': 'WD10M44530W_03786S-00_BPT.pdf'}, page_content='Português 33 Operações Somente secagem de roupas 2 3 1 4 Você pode secar suas roupas diretamente ou secar as roupas lavadas de acordo com os seguintes procedimentos 1 Pressione LigaDesliga para ligar a lavadora de roupas 2 Use o Seletor de Ciclos para selecionar o ciclo de secagem apropriado 3 Pressione a tecla Nível de Secagem para selecionar o modo de secagem apropriado 4 Pressione a tecla InícioPausa para iniciar a secagem OBSERVAÇÃO É possível economizar água e energia adicionando o ciclo'),
 Document(metadata={'cid': 1, 'page': 39, 'source': 'WD10M44530W_03786S-00_BPT.pdf'}, page_content='Secagem automática Gaveta Use esta opção para secar as roupas de algodão íntimas e de linho Modelo de 102kg6 kg Modelo de 85kg5 kg 3 kg Secagem manual Tempo de secagem Use esta opção para secar as roupas por um período de tempo específico dependendo das fibras quantidade de roupas e umidade 30  270

**Juntando textos em uma única string**

In [13]:
textos = '\n\n--\n\n'.join([d.page_content for d in context])

### Chamada do modelo LLM passando os parâmetros de contexto e a pergunta realizada

In [17]:
message = chain.invoke({'contexto': textos, 'pergunta': 'como devo fazer para realizar a secagem de roupas'})
print(message.content)

Para realizar a secagem de roupas, siga os seguintes passos:

1. Pressione o botão Liga/Desliga para ligar a lavadora de roupas.
2. Use o Seletor de Ciclos para selecionar o ciclo de secagem apropriado.
3. Pressione a tecla Nível de Secagem para escolher o modo de secagem desejado.
4. Pressione a tecla Início/Pausa para iniciar o processo de secagem.

Lembre-se de que é possível economizar água e energia ao adicionar o ciclo apropriado. Além disso, as opções de secagem, exceto a opção de Tempo de Secagem, detectam o peso das roupas para exibir um tempo de secagem mais preciso.
